<a href="https://colab.research.google.com/github/SSHDMU0001/WooriFis-AIU/blob/main/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

df_ratings  = pd.read_csv('acno3.csv')
df_goods  = pd.read_csv('master1.csv')
df_user_good_ratings = df_ratings.pivot(
    index='고객번호',
    columns='상품코드',
    values='평점'
).fillna(0)

df_user_good_ratings.head()

# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = df_user_good_ratings.to_numpy()

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)
matrix

matrix.shape

user_ratings_mean.shape

matrix_user_mean.shape

pd.DataFrame(matrix_user_mean, columns = df_user_good_ratings.columns).head()

# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 9)

print(U.shape)
print(sigma.shape)
print(Vt.shape)

sigma = np.diag(sigma)
sigma.shape

sigma[0]

sigma[1]

# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_good_ratings.columns)
df_svd_preds.head()

df_svd_preds.shape

def recommend_goods(df_svd_preds, user_id, ori_goods_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.고객번호 == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_goods_df, on = '상품코드').sort_values(['평점'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_goods_df[~ori_goods_df['상품코드'].isin(user_history['상품코드'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = '상품코드')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations
already_rated, predictions = recommend_goods(df_svd_preds, 1, df_goods, df_ratings, 10)
already_rated.head(10)

predictions

(10, 9)
(9,)
(9, 10)


,상품코드,상품명,상품종류,이자지급방법,Predictions
3,4,우리 첫거래우대 정기예금,정기예금,만기일시단리,4.440892e-16
1,2,모이면 금리가 올라가는 예금,정기예금,만기일시단리,5.551115e-17
2,3,우리 웰리치100 예금,정기예금,월이자지급식,5.551115e-17
4,5,WON플러스 예금,정기예금,만기일시단리,5.551115e-17
0,1,WON예금,정기예금,만기일시단리,-2.220446e-16
